In [1]:
import numpy as np
import numpy.linalg as la
import pickle as pkl
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import collections as mc
from scipy.spatial.distance import cdist
from scipy.optimize import linprog
from scipy.linalg import block_diag
from dataclasses import dataclass
from copy import copy, deepcopy
#from numba import prange, guvectorize, float64
import math
import pandas as pd

In [2]:
## MODEL PARAMETERS
radius = 1.         # Radius of cells
a = 6              # ATP required per fixed C
b = 5              # ATP per C in respiration
c = 4/10           # ATP produced per photon  
d = 53              # ATP per g of DW of biomass
xi = 42             # mmol of C /g of DW


In [4]:
def calculate_distances(cells_pos, window_size=50, dim=2):
    num_cells = len(cells_pos)
    distances = np.identity(num_cells)
    for i,cell1 in enumerate(cells_pos):
        for j,cell2 in enumerate(cells_pos):
            if i==j:
                distances[i][j] = window_size*10000
            else:
                if dim == 3: 
                    dx = abs(cell2[0] - cell1[0])
                    if dx > window_size/2:
                        dx = window_size - dx
                    dy = abs(cell2[1] - cell1[1])
                    if dy > window_size/2:
                        dy = window_size - dy
                    dz = abs(cell2[2] - cell1[2])
                    if dz > window_size/2:
                        dz = window_size - dz
                    distances[i][j] = math.sqrt(dx ** 2 + dy ** 2 + dz ** 2)
                else: 
                    distances[i][j] = math.sqrt((cell2[0] - cell1[0]) ** 2 + (cell2[1] - cell1[1]) ** 2 +
                                                (cell2[2] - cell1[2]) ** 2)
    return distances

            
def place_cells(num_cells, window_size=50, overlap=False, dim=2):
    min_sep = 2*radius
    rng = np.random.default_rng()
    cells_pos = rng.random((1, dim)) * (window_size)
    while len(cells_pos)<num_cells:
        new_pos =  rng.random((1, dim)) * (window_size)
        if not overlap: 
            cells_pos_new = np.vstack((cells_pos, new_pos))
            distances = calculate_distances(cells_pos_new, window_size, dim)
            if all(np.concatenate(distances)>min_sep):
                print(len(distances))
                cells_pos = cells_pos_new
            #else:
                #print(distances)
        else:
            cells_pos_new = np.vstack((cells_pos, new_pos))
            cells_pos = cells_pos_new
            
    return cells_pos



def A_matrix(cells_pos, window_size, dim):
    distances = calculate_distances(cells_pos, window_size, dim)
    num_cells = cells_pos.shape[0]
    A = np.identity(num_cells)
    for i in range(0,num_cells):
        for j in range(0,num_cells):
            if i!=j:
                A[i][j] = radius/distances[i][j]
    
    return A



In [5]:
for i in range(0,3):
    cells_pos = place_cells(746,window_size= 100,overlap=False, dim=3)
    distances = calculate_distances(cells_pos)
    np.fill_diagonal(distances, radius)
    pd.DataFrame(cells_pos).to_csv('cells_pos_configuration_%s.dat' %i, sep = '\t', header=False, index=False)

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27

640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746


In [7]:
## MATRIX GENERATION

n_cells = 746

# First block for inorganic carbon
N1 = []
for i in range(n_cells): 
    n = []
    for j in range(0,3*n_cells):
        if j == i*3:
            n.append(1.)
        else: 
            n.append(0.)
    N1.append(n)
N1 = np.array(N1)

# second block for oxygen
N2 = []
for i in range(n_cells): 
    n = []
    for j in range(0,3*n_cells):
        if j == i*3+1:
            n.append(1.)
        else: 
            n.append(0.)
    N2.append(n)
N2 = np.array(N2)

# third block for photons
N3 = []
for i in range(n_cells): 
    n = []
    for j in range(0,3*n_cells):
        if j == i*3+2:
            n.append(1.)
        else: 
            n.append(0.)
    N3.append(n)
N3 = np.array(N3)

#    u_C (inorg)    u_o     gamma     lambda
# f_ATP = - a*u_c + b*u_o + c*gamma - d*lambda
# forth block for ATP generation
a = 6
b = 5
c = 4/10
d = 53
N4 = []
for i in range(n_cells): 
    n = []
    for j in range(0,3*n_cells):
        if j == i*3:
            n.append(-a)
        elif j == i*3+1:
            n.append(b)
        elif j == i*3+2:
            n.append(c)
        else: 
            n.append(0.)
    N4.append(n)
N4 = np.array(N4)

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [18]:
results_file_name = '../results/standard_C_N_growth.xlsx'
std = pd.read_excel(results_file_name)

# Loading cell positions for the different configurations: 
for conf in range(0,3):
    cell_pos = np.loadtxt('cells_pos_configuration_%s.dat' %conf)
    distances = calculate_distances(cell_pos)
    np.fill_diagonal(distances, radius)
    
    # organic_C = -u_c + u_o + xi*lambda
    # 5th block for organic_C (constraints depend on spatial distribution)
    N5 = []
    for i in range(n_cells): 
        n = []
        for j in range(0,n_cells):
            n += [1, -1, 0]
        N5.append(n)
    N5 = np.array(N5)

    distances_ext = []    
    for i in distances:
        d = []
        for j in i:
            for r in range(3):
                d.append(j)
        distances_ext.append(d)
    distances_ext = np.array(distances_ext)

    N5 = N5/distances_ext * radius
    for i,cell in enumerate(N5): 
        if i == 0:
            N5[i,0] = 1
            N5[i,1] = -1
            N5[i,2] = 0
        else: 
            for j in range(n_cells*3): 
                if j == i*3:
                    N5[i,j] = 1
                elif j == i*3+1:
                    N5[i,j] = -1
                elif j == i*3+2:
                    N5[i,j] = 0
                elif j == i*3+3:
                    N5[i,j] = xi          
    N5 = np.around(N5, decimals=4)

    # Last row to constrain the sum of organic_C to be net release
    N6 = []
    for j in range(0,n_cells):
        N6 += [1, -1, 0]
    N6 = np.array([N6])
    
    # Matrix:
    matrix = np.concatenate([N1,N2,N3,N4,N5,N6])
    np.savetxt('matrix_configuration_%s.dat' %conf, matrix, delimiter='\t', fmt='%10.4f'  )
    
    ## BOUNDS FILE: 
    b1_min = [0 for i in range(n_cells)]
    b2_min = [0 for i in range(n_cells)]
    b3_min = [0 for i in range(n_cells)]
    b4_min = []
    for cell in std.iterrows():
        b4_min.append(53*cell[1]['growth_rate'])
    b5_min = []
    i = 0
    for cell_i in std.iterrows():
        min_val = 42*cell_i[1]['growth_rate']
        j = 0
        for cell_j in std.iterrows():
            if i != j:
                min_val += 42*cell_j[1]['growth_rate']/distances[i][j]*radius
            j += 1
        i+= 1
        b5_min.append(min_val)
    b6_min = [42*(np.sum(std['growth_rate']))]
    
    b1_max = []
    for cell in std.iterrows():
        b1_max.append(cell[1]['u_C']) 
    b2_max = []
    for cell in std.iterrows():
        b2_max.append(cell[1]['u_C']-42*cell[1]['growth_rate'])   
    b3_max = [9 for i in range(n_cells)]
    b4_max = [1000 for i in range(n_cells)]
    b5_max = [1000 for i in range(n_cells)]
    b6_max = [1000]
    
    b_min = b1_min+b2_min+b3_min+b4_min+b5_min+b6_min
    b_max = b1_max+b2_max+b3_max+b4_max+b5_max+b6_max
    bounds = np.array([b_min, b_max]).transpose()
    np.savetxt('bounds_configuration_%s.dat' %conf, bounds, delimiter='\t', fmt='%10.3f'  )